## 주식 가격 수집 및 저장 (대성에너지)
+ 저장된 데이터 마지막 일자의 다음날부터 현재일 기준 직전일까지 데이터를 일괄로 가져와서 저장
+ 작성: 임경호

#### 0. 데이터를 가져올 날짜 설정

In [1]:
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect
import myutil

stock_code = '117580'   # 대성에너지 stock code

conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

query = f'SELECT tr_date FROM stock_price where stock_code = {stock_code} ORDER BY tr_date DESC LIMIT 1'
cur.execute(query)

row = cur.fetchone()
last_date = row[0]     # 저장된 데이터의 마지막 일자

conn.close()

start_date = myutil.get_next_day(last_date)
end_date = myutil.get_previous_day()

In [2]:
stock_code, last_date, start_date, end_date

('117580', '20230907', '20230908', '20230910')

#### 1. 주가 데이터 가져오기

In [3]:
from pykrx import stock

# 특정기간 특정 종목의 OHLCV
ohlcv = stock.get_market_ohlcv(start_date, end_date, stock_code)

In [4]:
ohlcv.shape

(1, 7)

In [5]:
ohlcv

,시가,고가,저가,종가,거래량,거래대금,등락률
날짜,,,,,,,
2023-09-08,7510,7950,7350,7870,514912,3980204530,4.93


#### 2. 데이터 변환 및 전처리

In [6]:
df_ohlcv = ohlcv.reset_index()
df_ohlcv

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률
0,2023-09-08,7510,7950,7350,7870,514912,3980204530,4.93


In [7]:
# 날짜 형식 변환 => 문자열 yyyymmdd
df_ohlcv['날짜'] = df_ohlcv['날짜'].astype('str')
df_ohlcv['날짜'] = df_ohlcv['날짜'].str.replace('-', '')

In [8]:
df_ohlcv

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률
0,20230908,7510,7950,7350,7870,514912,3980204530,4.93


#### 3. DB 테이블에 저장

In [9]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_ohlcv.itertuples():
    sql = "insert into stock_price (stock_code, tr_date, open, high, low, close, volume, tr_amount, fluctuation_rate) \
                            values (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (stock_code, row[1], row[2], row[3], row[4], row[5], row[6], row[7], round(row[8],2)))
conn.commit()
conn.close()